In [2]:
import transformers
import torch
import datasets
from torch.utils.data import DataLoader
from functools import partial
from torch import nn
import dataclasses

import sys
sys.path.append('..')
import token_data
import micro_model

@dataclasses.dataclass
class TesterConfig:
    batch_size: int = 10
    max_seq_len: int = 1024
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

class Tester:
    def __init__(self, model, dataset, tokenizer, loss, config: TesterConfig = None):
        self.tokenizer = tokenizer
        self.model = model
        self.dataset = dataset
        self.config = config if config is not None else TesterConfig()
        self.dataloader = DataLoader(self.dataset, batch_size=self.config.batch_size, num_workers=1, collate_fn=partial(
            token_data.data_collator, self.tokenizer, self.config.max_seq_len))
        self.model = self.model.to(self.config.device)
        self.model.eval()
        self.model.to(torch.bfloat16)
        self.loss = loss

    @torch.no_grad()
    def test(self):
        i = 0
        for batch in self.dataloader:
            batch["input_ids"] = batch["input_ids"].to(self.config.device)
            batch["labels"] = batch["labels"].to(self.config.device)
            i += 1
            logits = self.model.forward(batch["input_ids"])
            if not isinstance(logits, torch.Tensor):
                logits = logits.logits
            loss = self.loss(logits.view(-1, logits.size(-1)), batch["labels"].view(-1))
            print(f"Loss: {loss}")
            if i == 10:
                break

/home/tl2020/micro/.venv/lib64/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = datasets.load_dataset(path="cerebras/SlimPajama-627B", split='test', trust_remote_code=True, streaming=True)

In [3]:
loss = nn.CrossEntropyLoss()

In [4]:

tokenizer = transformers.AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.1")
model = transformers.AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.1")

In [5]:
tester1 = Tester(model, dataset, tokenizer, loss)
tester1.test()

Loss: 3.0625
Loss: 2.484375
Loss: 3.453125
Loss: 3.328125
Loss: 2.84375
Loss: 2.953125
Loss: 3.515625
Loss: 2.65625
Loss: 3.28125
Loss: 3.03125


In [66]:
tokenizer = token_data.load_tokenizer()

model = micro_model.get_model().to('cpu').to(torch.bfloat16)
# ckpt = torch.load('/home/tl2020/micro/micro_model.pth')
# ckpt = torch.load("/home/tl2020/micro/micro_lm_sft/8bh1d845/checkpoints/epoch=2-step=1417.ckpt")
ckpt = torch.load("/home/tl2020/micro/micro_model_sft.pth")
model.load_state_dict(ckpt)
# print(ckpt["state_dict"].keys())
# print({ k[6:]: v for k, v in ckpt["state_dict"].items() })
# model.load_state_dict({ k[6:]: v for k, v in ckpt["state_dict"].items() })
# model.load_state_dict(ckpt)
# tester2 = Tester(model, dataset, tokenizer, loss)
# tester2.test()

/tmp/ipykernel_3571887/4136072741.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load("/home/tl2020/micro/micro_model_sft.pth")


<All keys matched successfully>

In [67]:
from chat_wrapper import ChatWrapper, GenerationConfig
chat = ChatWrapper(model, tokenizer, torch.bfloat16, "cpu")


In [68]:
text = """
<|user|>
What is the abstract of Quentin's next paper?
</s>
<|assistant|>
"""
print(chat.generate(text, GenerationConfig(256))[0])


Abstract:
Quentin's next paper, titled "Creating a Diverse Perspective in Environmental Issues: An Interdisciplinary Approach," examines the effects of climate change on different ecosystems, including forests, rivers, and the seas. The paper explores how environmental change is influenced by environmental factors such as temperature, precipitation, and moisture, and how these factors affect human health, ecosystem balance, and ecological stability.

The paper explores how climate change can affect ecosystem health, such as altering weather patterns, changing nutrient availability, and reducing heat loss, among other factors. The paper explores how different species have evolved to adapt to these changes, and how environmental factors can impact species' survival. The paper also examines the relationship between climate change and environmental change, which affects the natural resources and ecosystems that depend on it.

The paper concludes with a discussion of the implications of cli

In [13]:

print(chat.generate("Some of the most glorious historical attractions in Spain date from the period of Muslim rule, including The"
"Mezquita, built as the Great Mosque of Cordoba and the Medina Azahara, also in C ´ ordoba and now in ruins but ´"
"still visitable as such and built as the Madinat al-Zahra, the Palace of al-Andalus; and the Alhambra in Granada, a" , 
GenerationConfig(512))[0])

 World Heritage Site. From this great historical relic you will find some of the best museums in Spain, including the Sainte Tinta Museum, which houses one of the most important and famous religious collections in Spain and several more at the same time.
The Mezquita is considered to be one of the most important monuments of Spanish Culture and is only 1.2 km from Cordoba and has 10 museums with collections from the Middle Ages. It is among the best preserved monuments in Spain with over 100 of them surviving in the years 1845-1895.
Alberta's three major monuments - the House of Mosques, the Cathedral of El Arbol and the Main Mosque are covered with a magnificent collection of wood, polychrome and pendent. Alberta's museums include the National Library and the Bloc de Santillana in the city, which is an extraordinary collection of sculptures, drawings and many more.
Visit Almota's famous Archaeological Museum to see two of the oldest tombs in Almería. The museum houses more than 100 co

In [15]:
text = """
The 2008 Summer Olympics torch relay was run from March 24 until August 8, 2008, prior to the 2008 Summer
Olympics, with the theme of “one world, one dream”. Plans for the relay were announced on April 26, 2007, in
Beijing, China. The relay, also called by the organizers as the “Journey of Harmony”, lasted 129 days and carried
the torch 137,000 km (85,000 mi) – the longest distance of any Olympic torch relay since the tradition was started
ahead of the 1936 Summer Olympics.
After being lit at the birthplace of the Olympic Games in Olympia, Greece on March 24, the torch traveled to the Panathinaiko Stadium in Athens, and then to Beijing, arriving on March 31. From Beijing, the torch was
following a route passing through six continents. The torch has visited cities along the Silk Road, symbolizing
ancient links between China and the rest of the world. The relay also included an ascent with the flame to the top of
Mount Everest on the border of Nepal and Tibet, China from the Chinese side, which was closed specially for the
event.
Q: What was the theme
A: “one world, one dream”.
Q: What was the length of the race?
A: 137,000 km
Q: Was it larger than previous ones?
A: No.
Q: Where did the race begin?
"""

print(chat.generate(text, GenerationConfig(512))[0])


A: China began the race with the founding of the Beijing Olympics in 1909, 2000 and 2002,
for the record. The event, known as the "Little Miss America" (aka ​"Golden Time"), was conceived to create the first-ever Olympic torch relay, taking place in Beijing from February 23 to March 13, 2008, when the
golden hour was measured.
Q: What were the route goals?
A: The city of Beijing
Q: What was the exact day when the torch relay was launched?
A: It was on March 24, 2007.
Q: Did you take the same roads, climbs, climbs, etc. in Beijing?
A: The only problems I had with the trip were the weather and the lack of fuel. So I had to drive on one of these, and they were almost a mile from my house.
Q: How did you plan this?
A: The track itself is no more than a single parkway, but the lane you take, it was supposed to be the single most important place for the events of the Olympics. So this meant that the riders (on the road) couldn't get up and down, and then the trucks could come and go. So I ha